In this notebook, I'll look to persist the basic "enum tables" to my fbref football schema db. These tables include:
* country
* seasons
* teams
* competitions

# Imports

In [1]:
import os
os.chdir("../../")

In [2]:
import pandas as pd
from src.fbref.fbref_class import FBref
from src.utility.sql.fetch_and_persist import (
    create_db_engine,
    query_db,
    persist_to_db,
)

# Create DB connection

In [3]:
engine = create_db_engine(
    db_user = os.environ.get("POSTGRES_USER"), 
    db_password = os.environ.get("POSTGRES_PASS"), 
    db_host = os.environ.get("POSTGRES_HOST"), 
    db_port = os.environ.get("POSTGRES_PORT"), 
    db_name = os.environ.get("POSTGRES_DB")
)

# Fetch and persist tables

### Instantiate fbref class

In [4]:
fb = FBref()

### Get country df

In [5]:
player_standard_df = fb.get_big5_player_stats(table_type='standard', season_name='2022-2023')

In [6]:
country_list = player_standard_df.sort_values(by=['country'])['country'].unique()
country_df = pd.DataFrame(country_list, columns=['country_code'])

persist country df

In [7]:
# country df
persist_to_db(
    df_to_persist = country_df, 
    table_name = 'country',
    schema_name = 'fbref',
    engine = engine
)

Data Persisted


### Get seasons df

Let's get comeptition links for the top five leageus where the competition links are used as input to get seasons for that specific competition

In [8]:
big5_df = fb.get_big_5_leagues()

competition_link_dict = dict(
    zip(
        big5_df['Competition Name'],
        big5_df['competition_link']
    )
)

Create season df using seasons from Premier League

In [9]:
prem_seasons_df = fb.get_competition_seasons(competition_link_dict['Premier League'])
season_list = list(prem_seasons_df.Season)

season_df = pd.DataFrame(season_list, columns=['season_name'])

persist seasons df

In [10]:
# season df
persist_to_db(
    df_to_persist = season_df, 
    table_name = 'seasons',
    schema_name = 'fbref',
    engine = engine
)


Data Persisted


### Get teams df

Get teams from top five countries

In [11]:
teams_df = pd.concat(
    [fb.get_teams_per_country(country) for country in ["England", "France", "Germany", "Spain", "Italy"] ]
)

In [12]:
teams_df = (
    teams_df
    .rename(
        columns={
            "Gender" : "gender",
            "Squad" : "team_name",
            "team_id" : "team_fb_ref_id",
        }
    )
)[
    [
        'gender', 
        'team_name', 
        'team_fb_ref_id'
    ]
]

persist teams df

In [13]:
# teams df
persist_to_db(
    df_to_persist = teams_df, 
    table_name = 'teams',
    schema_name = 'fbref',
    engine = engine
)

Data Persisted


### Get competition df

fetch country df from db

In [14]:
db_country_df = query_db('SELECT * FROM fbref.country', engine)

Prepare comeptition table

In [15]:
competition_df = big5_df[big5_df.competition_id.isin(['9', '12', '13', '20', '11'])]

competition_df = (
    competition_df
    # clean country values to codes
    .assign(
        Country = competition_df.Country.apply(lambda x: x.split(' ')[1])
    )
    # merge with country df to get country id
    .merge(db_country_df, left_on='Country', right_on='country_code')
    .rename(
        columns = {
            'Gender': 'gender',
            'Competition Name':'competition_name',
            'competition_id':'competition_fb_ref_id', 
        }
    )
)


In [16]:
competition_df = competition_df[
    [
        'country_id',
        'gender',
        'competition_name',
        'competition_link',
        'competition_fb_ref_id', 
    ]
]

In [17]:
# teams df
persist_to_db(
    df_to_persist = competition_df, 
    table_name = 'competitions',
    schema_name = 'fbref',
    engine = engine
)

Data Persisted
